# 03 - Parser Benchmark

Comparaison des méthodes d'extraction PDF :
- **pdfplumber** : Parser mécanique (extraction de tableaux)
- **Mistral OCR** : Vision model cloud (mistral-ocr-2503)
- **Docling** : Vision model local open source (IBM)

## Objectif
Mesurer la précision d'extraction des bulletins scolaires vs ground truth.

In [1]:
# ruff: noqa: E402
import json
import os
import sys
import time
from pathlib import Path

# Auto-détecter project_root
current = Path.cwd()
while current != current.parent:
    if (current / "pyproject.toml").exists():
        project_root = current
        break
    current = current.parent

sys.path.insert(0, str(project_root))

from dotenv import load_dotenv

load_dotenv(project_root / ".env")

# Paths
DATA_DIR = project_root / "data"
RAW_DIR = DATA_DIR / "raw"
GROUND_TRUTH_PATH = DATA_DIR / "ground_truth" / "chiron_ground_truth.json"

print(f"Project root: {project_root}")
print(f"PDFs: {list(RAW_DIR.glob('*.pdf'))}")

Project root: c:\Users\Florent\Documents\data_science\chiron
PDFs: [WindowsPath('c:/Users/Florent/Documents/data_science/chiron/data/raw/ELEVE_A.pdf'), WindowsPath('c:/Users/Florent/Documents/data_science/chiron/data/raw/ELEVE_B.pdf'), WindowsPath('c:/Users/Florent/Documents/data_science/chiron/data/raw/ELEVE_C.pdf'), WindowsPath('c:/Users/Florent/Documents/data_science/chiron/data/raw/ELEVE_D.pdf')]


In [2]:
# Charger ground truth
with open(GROUND_TRUTH_PATH, encoding="utf-8") as f:
    ground_truth = json.load(f)

# Index par eleve_id pour comparaison facile
gt_by_eleve = {e["eleve_id"]: e for e in ground_truth["eleves"]}
print(f"Ground truth: {list(gt_by_eleve.keys())}")

Ground truth: ['ELEVE_A', 'ELEVE_B', 'ELEVE_C', 'ELEVE_D']


## 1. Parser pdfplumber (actuel)

In [ ]:
from src.document import ParserType, get_parser

parser = get_parser(ParserType.PDFPLUMBER)

pdfplumber_results = {}
pdfplumber_times = {}

for pdf_path in sorted(RAW_DIR.glob("*.pdf")):
    eleve_id = pdf_path.stem  # ELEVE_A, ELEVE_B, etc.

    start = time.perf_counter()
    try:
        eleves = parser.parse(pdf_path)
        pdfplumber_results[eleve_id] = eleves[0] if eleves else None
    except Exception as e:
        pdfplumber_results[eleve_id] = f"ERROR: {e}"
    pdfplumber_times[eleve_id] = time.perf_counter() - start

    print(f"{eleve_id}: {pdfplumber_times[eleve_id]:.2f}s")

In [4]:
# Afficher un exemple de résultat pdfplumber
exemple = pdfplumber_results.get("ELEVE_A")
if exemple and not isinstance(exemple, str):
    print(f"Nom: {exemple.nom}")
    print(f"Prénom: {exemple.prenom}")
    print(f"Classe: {exemple.classe}")
    print(f"Matières extraites: {len(exemple.matieres)}")
    for m in exemple.matieres[:]:
        print(f"  - {m.nom}: {m.moyenne_eleve}")
else:
    print(f"Erreur ou pas de résultat: {exemple}")

Nom: None
Prénom: None
Classe: None
Matières extraites: 0


## 2. Mistral OCR

In [5]:
import base64

from mistralai import Mistral

# Init client
mistral_api_key = os.getenv("MISTRAL_OCR_API_KEY")
if not mistral_api_key:
    raise ValueError("MISTRAL_OCR_API_KEY non configurée dans .env")

client = Mistral(api_key=mistral_api_key)
mistral_model = "mistral-ocr-latest"
print("Client Mistral initialisé")

Client Mistral initialisé


In [6]:
def encode_pdf(pdf_path):
    """Encode the pdf to base64."""
    try:
        with open(pdf_path, "rb") as pdf_file:
            return base64.b64encode(pdf_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {pdf_path} was not found.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

In [7]:
def extract_with_mistral_ocr(pdf_path: Path) -> dict:
    """Extrait le contenu d'un PDF avec Mistral OCR."""
    # Getting the base64 string
    base64_pdf = encode_pdf(pdf_path)

    # Call the OCR API
    pdf_response = client.ocr.process(
        model=mistral_model,
        document={
            "type": "document_url",
            "document_url": f"data:application/pdf;base64,{base64_pdf}"
        },
        include_image_base64=True
        )

    # Convert response to JSON format
    response_dict = json.loads(pdf_response.model_dump_json())

    return response_dict

In [8]:
# Extraire tous les PDFs avec Mistral OCR
mistral_results = {}
mistral_times = {}

for pdf_path in sorted(RAW_DIR.glob("*.pdf")):
    eleve_id = pdf_path.stem

    start = time.perf_counter()
    try:
        result = extract_with_mistral_ocr(pdf_path)
        mistral_results[eleve_id] = result
    except Exception as e:
        mistral_results[eleve_id] = f"ERROR: {e}"
    mistral_times[eleve_id] = time.perf_counter() - start

    print(f"{eleve_id}: {mistral_times[eleve_id]:.2f}s")

ELEVE_A: 1.96s
ELEVE_B: 2.08s
ELEVE_C: 1.72s
ELEVE_D: 1.90s


In [9]:
print(json.dumps(mistral_results["ELEVE_A"], indent=4)[0:1000]) # check the first 1000 characters

{
    "pages": [
        {
            "index": 0,
            "markdown": "# College Test\nBULLETIN SCOLAIRE\n\n\u00c9l\u00e8ve : ELEVE_A\nGenre : Fille\nAbsences : 4 demi-journ\u00e9es (justifi\u00e9es)\nEngagements : D\u00e9l\u00e9gu\u00e9e titulaire\n\n|  Mati\u00e8re | \u00c9l\u00e8ve / Classe | Appr\u00e9ciation  |\n| --- | --- | --- |\n|  Anglais LV1 | 15.21 / 10.83 | Bons r\u00e9sultats. ELEVE_A fournit un travail r\u00e9gulier et s\u00e9rieux \u00e0 la maison tout comme en classe. L'attitude est toujours positive et constructive. Poursuivez ainsi!  |\n|  Arts Plastiques | 15.00 / 14.92 | Bon ensemble, bilan satisfaisant, continuez ainsi en restant concentr\u00e9e.  |\n|  EPS | 8.00 / 12.79 | Bilan tr\u00e8s insuffisant, ELEVE_A n'a pas r\u00e9ussi \u00e0 s'orienter avec efficacit\u00e9. Elle a beaucoup march\u00e9 et s'est dispers\u00e9e avec son groupe. Son travail a manqu\u00e9 de s\u00e9rieux et d'implication dans les d\u00e9fis \u00e0 relever en course d'orientation. De si

## 3. Docling (IBM - Open Source Local)

In [10]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
from docling.document_converter import DocumentConverter, PdfFormatOption

pipeline_options = PdfPipelineOptions(do_table_structure=True)
pipeline_options.table_structure_options.mode = TableFormerMode.ACCURATE  # use more accurate TableFormer model

# Init converter (première exécution télécharge les modèles)
docling_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)
print("Docling converter initialisé")

c:\Users\Florent\Documents\data_science\chiron\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Docling converter initialisé


In [11]:

# Extraire tous les PDFs avec Docling
docling_results = {}
docling_times = {}

for pdf_path in sorted(RAW_DIR.glob("*.pdf")):
    eleve_id = pdf_path.stem

    start = time.perf_counter()
    try:
        result = docling_converter.convert(str(pdf_path))
        # Export direct en markdown (comme Mistral OCR)
        docling_results[eleve_id] = result.document.export_to_markdown()
    except Exception as e:
        docling_results[eleve_id] = f"ERROR: {e}"
    docling_times[eleve_id] = time.perf_counter() - start

    print(f"{eleve_id}: {docling_times[eleve_id]:.2f}s")

[INFO] 2026-01-22 10:38:33,377 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-22 10:38:33,389 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2026-01-22 10:38:33,402 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\Florent\Documents\data_science\chiron\.venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-22 10:38:33,404 [RapidOCR] main.py:50: Using C:\Users\Florent\Documents\data_science\chiron\.venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-22 10:38:33,848 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-22 10:38:33,849 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2026-01-22 10:38:33,851 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\Florent\Documents\data_science\chiron\.venv\Lib\site-packages\rapidocr\models\ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-22 10:38:33,852 [RapidOCR] main.py:50: Using C:\Users\Florent\Documents\data_scie

ELEVE_A: 22.32s
ELEVE_B: 4.44s
ELEVE_C: 4.60s
ELEVE_D: 4.85s


In [12]:
print(docling_results["ELEVE_A"])

## College Test BULLETIN SCOLAIRE

Élève

: ELEVE\_A

Genre

: Fille

Absences

: 4 demi-journées (justifiées)

Engagements

: Déléguée titulaire

| Matière                 | Élève / Classe   | Appréciation                                                                                                                                                                                                                                                               |
|-------------------------|------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Anglais LV1             | 15.21 / 10.83    | Bons résultats. ELEVE_A fournit un travail régulier et sérieux à la maison comme en classe. L'attitude est toujours positive et constructive. ainsi!                          

## 4. Normalisation des extractions

Convertir les résultats des trois parsers en format uniforme pour comparaison.

In [13]:
import re

import pandas as pd


def extract_key_value(text: str, key: str) -> str | None:
    """Extrait une valeur pour une clé donnée.

    Gère deux formats :
    - 'Key : Value' (même ligne)
    - 'Key\\n: Value' (ligne séparée, format Docling)
    """
    if not text:
        return None
    # Format standard: Key : Value
    pattern = rf"{key}\s*:\s*([^\n]+)"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    # Format Docling: Key\n: Value
    pattern_docling = rf"{key}\s*\n\s*:\s*([^\n]+)"
    match = re.search(pattern_docling, text, re.IGNORECASE)
    return match.group(1).strip() if match else None


def extract_number(text: str) -> float | None:
    """Extrait le premier nombre d'un texte."""
    if not text:
        return None
    match = re.search(r"(\d+[,.]?\d*)", text)
    if match:
        return float(match.group(1).replace(",", "."))
    return None


def extract_note_pair(text: str) -> tuple[float | None, float | None]:
    """Extrait une paire de notes (élève / classe)."""
    if not text:
        return None, None
    match = re.search(r"(\d+[,.]?\d*)\s*[/|]\s*(\d+[,.]?\d*)", text)
    if match:
        return float(match.group(1).replace(",", ".")), float(match.group(2).replace(",", "."))
    # Fallback: juste un nombre
    num = extract_number(text)
    return num, None


def parse_raw_tables(tables: list) -> list[dict]:
    """Parse les tables brutes en liste de matières."""
    if not tables:
        return []

    matieres = []
    for table in tables:
        for row in table:
            if not row or len(row) < 2:
                continue

            # Colonne 0: nom (nettoyer)
            nom = " ".join((row[0] or "").split()).strip()
            if not nom:
                continue

            # Colonne 1: notes
            note_text = " ".join((row[1] or "").split()).strip()
            moy_eleve, moy_classe = extract_note_pair(note_text)

            # Ignorer les lignes sans notes (headers)
            if moy_eleve is None:
                continue

            # Dernière colonne non vide: appréciation
            appreciation = ""
            for cell in reversed(row[2:]):
                text = " ".join((cell or "").split()).strip()
                if text and len(text) > 5:
                    appreciation = text
                    break

            matieres.append({
                "nom": nom,
                "moy_eleve": moy_eleve,
                "moy_classe": moy_classe,
                "appreciation": appreciation,
            })

    return matieres


def parse_markdown_table(md: str) -> list[dict]:
    """Parse un tableau markdown en liste de matières."""
    skip_values = {"Matière", ":--:", "---", ""}
    matieres = []

    for r in re.findall(r"\|\s*([^|]+)\s*\|\s*([^|]+)\s*\|\s*([^|]*)\s*\|", md):
        nom = r[0].strip()
        if nom in skip_values or "Élève" in nom or "lève" in nom:
            continue

        moy_eleve = float(m.group(1)) if (m := re.search(r"([\d.]+)\s*/", r[1])) else None
        moy_classe = float(m.group(1)) if (m := re.search(r"/\s*([\d.]+)", r[1])) else None

        # Ignorer les lignes sans notes
        if moy_eleve is None:
            continue

        matieres.append({
            "nom": nom,
            "moy_eleve": moy_eleve,
            "moy_classe": moy_classe,
            "appreciation": r[2].strip() if len(r) > 2 else "",
        })

    return matieres


def normalize(source: str, data) -> dict | None:
    """Normalise n'importe quelle source en dict unifié."""
    if data is None or isinstance(data, str) and data.startswith("ERROR"):
        return None

    if source == "ground_truth":
        return {
            "eleve_id": data["eleve_id"],
            "genre": data["genre"],
            "absences": data["absences_demi_journees"],
            "engagements": ", ".join(data.get("engagements", [])),
            "matieres": [
                {"nom": m["nom"], "moy_eleve": m["moyenne_eleve"],
                 "moy_classe": m["moyenne_classe"], "appreciation": m.get("appreciation", "")}
                for m in data["matieres"]
            ],
            "moyenne_generale": data.get("moyenne_generale"),
        }

    if source == "pdfplumber":
        raw_text = data.raw_text or ""
        raw_tables = data.raw_tables or []

        return {
            "eleve_id": extract_key_value(raw_text, r"[ÉE]l[èe]ve"),
            "genre": extract_key_value(raw_text, "Genre"),
            "absences": int(n) if (n := extract_number(extract_key_value(raw_text, "Absences?"))) else None,
            "engagements": extract_key_value(raw_text, "Engagements?"),
            "matieres": parse_raw_tables(raw_tables),
            "moyenne_generale": extract_number(extract_key_value(raw_text, r"Moyenne\s+g[ée]n[ée]rale")),
        }

    if source == "mistral_ocr":
        md = data["pages"][0]["markdown"]
        return {
            "eleve_id": extract_key_value(md, r"[ÉE]l[èe]ve"),
            "genre": extract_key_value(md, "Genre"),
            "absences": int(n) if (n := extract_number(extract_key_value(md, "Absences?"))) else None,
            "engagements": extract_key_value(md, "Engagements?"),
            "matieres": parse_markdown_table(md),
            "moyenne_generale": extract_number(extract_key_value(md, r"Moyenne\s+g[ée]n[ée]rale")),
        }

    if source == "docling":
        # Docling retourne directement du markdown (string)
        md = data if isinstance(data, str) else ""
        return {
            "eleve_id": extract_key_value(md, r"[ÉE]l[èe]ve"),
            "genre": extract_key_value(md, "Genre"),
            "absences": int(n) if (n := extract_number(extract_key_value(md, "Absences?"))) else None,
            "engagements": extract_key_value(md, "Engagements?"),
            "matieres": parse_markdown_table(md),
            "moyenne_generale": extract_number(extract_key_value(md, r"Moyenne\s+g[ée]n[ée]rale")),
        }

    return None


# Normaliser tous les résultats
normalized = {"pdfplumber": {}, "mistral_ocr": {}, "docling": {}}

for eleve_id in gt_by_eleve.keys():
    normalized["pdfplumber"][eleve_id] = normalize("pdfplumber", pdfplumber_results.get(eleve_id))
    normalized["mistral_ocr"][eleve_id] = normalize("mistral_ocr", mistral_results.get(eleve_id))
    normalized["docling"][eleve_id] = normalize("docling", docling_results.get(eleve_id))

print("Données normalisées pour:")
for source, data in normalized.items():
    print(f"  - {source}: {list(data.keys())}")

Données normalisées pour:
  - pdfplumber: ['ELEVE_A', 'ELEVE_B', 'ELEVE_C', 'ELEVE_D']
  - mistral_ocr: ['ELEVE_A', 'ELEVE_B', 'ELEVE_C', 'ELEVE_D']
  - docling: ['ELEVE_A', 'ELEVE_B', 'ELEVE_C', 'ELEVE_D']


## 5. Export des extractions

Sauvegarder les extractions en markdown pour faciliter la comparaison visuelle.

In [14]:
BENCHMARK_DIR = DATA_DIR / "processed" / "benchmark-extraction"
BENCHMARK_DIR.mkdir(parents=True, exist_ok=True)


def export_to_markdown(data: dict, source: str, output_path: Path):
    """Exporte les données normalisées en markdown."""
    lines = [
        f"# {source} - {data.get('eleve_id', 'N/A')}",
        "",
        f"**Genre:** {data.get('genre', 'N/A')}",
        f"**Absences:** {data.get('absences', 'N/A')} demi-journées",
        f"**Engagements:** {data.get('engagements') or 'Aucun'}",
        "",
        "## Matières",
        "",
        "| Matière | Moy. Élève | Moy. Classe | Appréciation |",
        "|---------|------------|-------------|--------------|",
    ]

    for m in data.get("matieres", []):
        app = (m.get("appreciation") or "").replace("|", "\\|").replace("\n", " ")
        lines.append(f"| {m['nom']} | {m['moy_eleve']} | {m['moy_classe']} | {app} |")

    if data.get("moyenne_generale"):
        lines.extend(["", f"**Moyenne générale:** {data['moyenne_generale']}/20"])

    output_path.write_text("\n".join(lines), encoding="utf-8")


# Exporter pour tous les élèves
for eleve_id in gt_by_eleve.keys():
    # Ground truth
    gt_norm = normalize("ground_truth", gt_by_eleve[eleve_id])
    export_to_markdown(gt_norm, "Ground Truth", BENCHMARK_DIR / f"{eleve_id}_ground_truth.md")

    # pdfplumber
    if normalized["pdfplumber"].get(eleve_id):
        export_to_markdown(normalized["pdfplumber"][eleve_id], "pdfplumber", BENCHMARK_DIR / f"{eleve_id}_pdfplumber.md")

    # Mistral OCR
    if normalized["mistral_ocr"].get(eleve_id):
        export_to_markdown(normalized["mistral_ocr"][eleve_id], "Mistral OCR", BENCHMARK_DIR / f"{eleve_id}_mistral_ocr.md")

    # Docling
    if normalized["docling"].get(eleve_id):
        export_to_markdown(normalized["docling"][eleve_id], "Docling", BENCHMARK_DIR / f"{eleve_id}_docling.md")

    print(f"✓ {eleve_id} exporté")

print(f"\nFichiers exportés dans: {BENCHMARK_DIR}")

✓ ELEVE_A exporté
✓ ELEVE_B exporté
✓ ELEVE_C exporté
✓ ELEVE_D exporté

Fichiers exportés dans: c:\Users\Florent\Documents\data_science\chiron\data\processed\benchmark-extraction


## 6. Comparaison avec le Ground Truth

Fonction de comparaison générique pour les trois sources.

In [15]:
def compare_with_gt(extracted: dict, gt: dict, source: str) -> pd.DataFrame:
    """Compare les données extraites avec le ground truth.

    Args:
        extracted: Données normalisées extraites
        gt: Ground truth
        source: Nom de la source (pdfplumber, mistral_ocr)

    Returns:
        DataFrame avec les résultats de comparaison
    """
    if not extracted:
        return pd.DataFrame()

    comparisons = []
    gt_matieres = {m["nom"]: m for m in gt["matieres"]}

    for m in extracted["matieres"]:
        nom = m["nom"]
        gt_m = gt_matieres.get(nom, {})

        # Comparer moyennes
        moy_eleve_ok = m["moy_eleve"] == gt_m.get("moyenne_eleve")
        moy_classe_ok = m["moy_classe"] == gt_m.get("moyenne_classe")

        # Comparer appréciations (normaliser les espaces)
        app_ext = " ".join((m.get("appreciation") or "").split())
        app_gt = " ".join(gt_m.get("appreciation", "").split())
        app_ok = app_ext == app_gt

        comparisons.append({
            "source": source,
            "matiere": nom,
            "moy_eleve_ext": m["moy_eleve"],
            "moy_eleve_gt": gt_m.get("moyenne_eleve"),
            "moy_eleve_ok": "✅" if moy_eleve_ok else "❌",
            "moy_classe_ext": m["moy_classe"],
            "moy_classe_gt": gt_m.get("moyenne_classe"),
            "moy_classe_ok": "✅" if moy_classe_ok else "❌",
            "app_ok": "✅" if app_ok else "❌",
            "app_len_ext": len(app_ext),
            "app_len_gt": len(app_gt),
        })

    return pd.DataFrame(comparisons)


# Comparer toutes les sources pour tous les élèves
all_comparisons = []

for source_name, source_data in normalized.items():
    for eleve_id, extracted in source_data.items():
        df = compare_with_gt(extracted, gt_by_eleve[eleve_id], source_name)
        if not df.empty:
            df["eleve_id"] = eleve_id
            all_comparisons.append(df)

df_comparison = pd.concat(all_comparisons, ignore_index=True)
print(f"Total comparaisons: {len(df_comparison)} lignes")

Total comparaisons: 138 lignes


In [16]:
# Statistiques par source
stats_by_source = []

for source in df_comparison["source"].unique():
    df_src = df_comparison[df_comparison["source"] == source]
    total = len(df_src)

    stats_by_source.append({
        "source": source,
        "total_matieres": total,
        "moy_eleve_ok": (df_src["moy_eleve_ok"] == "✅").sum(),
        "moy_classe_ok": (df_src["moy_classe_ok"] == "✅").sum(),
        "app_ok": (df_src["app_ok"] == "✅").sum(),
    })

df_stats = pd.DataFrame(stats_by_source)
df_stats["moy_eleve_%"] = (df_stats["moy_eleve_ok"] / df_stats["total_matieres"] * 100).round(1)
df_stats["moy_classe_%"] = (df_stats["moy_classe_ok"] / df_stats["total_matieres"] * 100).round(1)
df_stats["app_%"] = (df_stats["app_ok"] / df_stats["total_matieres"] * 100).round(1)

print("=" * 60)
print("STATISTIQUES PAR SOURCE")
print("=" * 60)
print(df_stats[["source", "total_matieres", "moy_eleve_%", "moy_classe_%", "app_%"]].to_string(index=False))

STATISTIQUES PAR SOURCE
     source  total_matieres  moy_eleve_%  moy_classe_%  app_%
 pdfplumber              46        100.0         100.0  100.0
mistral_ocr              46        100.0         100.0  100.0
    docling              46        100.0         100.0   37.0


In [17]:
# Détail des appréciations qui ne matchent pas
df_errors = df_comparison[df_comparison["app_ok"] == "❌"][
    ["source", "eleve_id", "matiere", "app_len_ext", "app_len_gt"]
]
print(f"Appréciations différentes ({len(df_errors)}):\n")
if not df_errors.empty:
    print(df_errors.to_string(index=False))
else:
    print("Aucune erreur d'appréciation!")

Appréciations différentes (29):

 source eleve_id                 matiere  app_len_ext  app_len_gt
docling  ELEVE_A             Anglais LV1          148         164
docling  ELEVE_A                     EPS          266         291
docling  ELEVE_A      Éducation Musicale           65          75
docling  ELEVE_A                Français          169         184
docling  ELEVE_A Histoire-Géographie-EMC          142         158
docling  ELEVE_A           Mathématiques          188         204
docling  ELEVE_B             Anglais LV1          207         219
docling  ELEVE_B                     EPS           86          91
docling  ELEVE_B                Français          168         182
docling  ELEVE_B             Italien LV2          250         268
docling  ELEVE_B           Mathématiques          211         226
docling  ELEVE_B         Physique-Chimie           92         106
docling  ELEVE_B                     SVT          149         162
docling  ELEVE_B             Technologie   

## 7. Résumé comparatif

In [18]:
# Temps d'exécution
df_times = pd.DataFrame([
    {
        "eleve_id": eleve_id,
        "pdfplumber_time": pdfplumber_times.get(eleve_id),
        "mistral_ocr_time": mistral_times.get(eleve_id),
        "docling_time": docling_times.get(eleve_id),
    }
    for eleve_id in gt_by_eleve.keys()
])

print("Temps d'exécution (secondes):")
print(df_times.to_string(index=False))
print(f"\nMoyenne pdfplumber: {df_times['pdfplumber_time'].mean():.2f}s")
print(f"Moyenne Mistral OCR: {df_times['mistral_ocr_time'].mean():.2f}s")
print(f"Moyenne Docling: {df_times['docling_time'].mean():.2f}s")

Temps d'exécution (secondes):
eleve_id  pdfplumber_time  mistral_ocr_time  docling_time
 ELEVE_A         0.063753          1.960890     22.323617
 ELEVE_B         0.062796          2.079989      4.443762
 ELEVE_C         0.057298          1.724445      4.601950
 ELEVE_D         0.068962          1.897707      4.846486

Moyenne pdfplumber: 0.06s
Moyenne Mistral OCR: 1.92s
Moyenne Docling: 9.05s


In [19]:
# Tableau récapitulatif final
print("=" * 80)
print("RÉSUMÉ BENCHMARK")
print("=" * 80)

# Helper pour récupérer les stats d'une source
def get_stats(source):
    row = df_stats[df_stats["source"] == source]
    if row.empty:
        return {"moy_eleve_%": "N/A", "moy_classe_%": "N/A", "app_%": "N/A"}
    return row.iloc[0]

summary = pd.DataFrame({
    "Métrique": [
        "Temps moyen (s)",
        "Moyennes élève correctes",
        "Moyennes classe correctes",
        "Appréciations identiques",
    ],
    "pdfplumber": [
        f"{df_times['pdfplumber_time'].mean():.2f}",
        f"{get_stats('pdfplumber')['moy_eleve_%']}%",
        f"{get_stats('pdfplumber')['moy_classe_%']}%",
        f"{get_stats('pdfplumber')['app_%']}%",
    ],
    "mistral_ocr": [
        f"{df_times['mistral_ocr_time'].mean():.2f}",
        f"{get_stats('mistral_ocr')['moy_eleve_%']}%",
        f"{get_stats('mistral_ocr')['moy_classe_%']}%",
        f"{get_stats('mistral_ocr')['app_%']}%",
    ],
    "docling": [
        f"{df_times['docling_time'].mean():.2f}",
        f"{get_stats('docling')['moy_eleve_%']}%",
        f"{get_stats('docling')['moy_classe_%']}%",
        f"{get_stats('docling')['app_%']}%",
    ],
})

print(summary.to_string(index=False))

print("\n" + "=" * 80)
print("CONCLUSION")
print("=" * 80)
print("""
## Résultats du benchmark

**Mistral OCR** : 100% de précision sur notes et appréciations
- Interprète correctement la structure sémantique
- Post-traitement minimal requis

**pdfplumber** : 100% sur notes, nécessite post-traitement
- Extraction mécanique (pas de ML)
- Headers multi-lignes mal gérés sans normalisation
- Maintenance requise si le format change

**Docling (IBM)** : NON RETENU
- Bug connu : perte de mots aux retours à la ligne dans les cellules
- Malgré une table simple (3 colonnes, pas de fusion), le modèle
  TableFormer tronque le texte (ex: "Poursuivez ainsi!" → "ainsi!")
- Issues GitHub #1922, #2064 documentent ce problème

## Autres solutions évaluées (non testées)

Solutions locales open source considérées mais non retenues :
- **PDF-Extract-Kit / MinerU** : Complet mais complexe (modèles différents par type)
- **olmOCR** (Allen AI) : Prometteur mais installation lourde
- **PaddleOCR** (Baidu) : Bon sur tableaux mais expertise requise
- **Marker** : PDF→Markdown, moins mature

Ces solutions demandent une expertise OCR et une maintenance
qui dépassent le cadre du projet Chiron.

## Choix final : Mistral OCR

Raisons :
- Précision 100% sans post-traitement complexe
- API simple (5 lignes de code)
- Gère PDF et images de manière uniforme
- Robuste face aux variations de format
- Pas de maintenance de modèles ou règles spécifiques
- Coût négligeable (~0.001€/page, <1€/trimestre pour Chiron)

Le temps économisé sur la configuration et maintenance d'une solution
locale justifie largement le coût minime de l'API.
""")

RÉSUMÉ BENCHMARK
                 Métrique pdfplumber mistral_ocr docling
          Temps moyen (s)       0.06        1.92    9.05
 Moyennes élève correctes     100.0%      100.0%  100.0%
Moyennes classe correctes     100.0%      100.0%  100.0%
 Appréciations identiques     100.0%      100.0%   37.0%

CONCLUSION

## Résultats du benchmark

**Mistral OCR** : 100% de précision sur notes et appréciations
- Interprète correctement la structure sémantique
- Post-traitement minimal requis

**pdfplumber** : 100% sur notes, nécessite post-traitement
- Extraction mécanique (pas de ML)
- Headers multi-lignes mal gérés sans normalisation
- Maintenance requise si le format change

**Docling (IBM)** : NON RETENU
- Bug connu : perte de mots aux retours à la ligne dans les cellules
- Malgré une table simple (3 colonnes, pas de fusion), le modèle
  TableFormer tronque le texte (ex: "Poursuivez ainsi!" → "ainsi!")
- Issues GitHub #1922, #2064 documentent ce problème

## Autres solutions évaluées (non t